In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 17.8MB/s 


In [0]:
file = drive.CreateFile({'id': '1aLon9sOQd49HNV0-Deo2XMwf-49XlL5P'})
file.GetContentFile('data.zip')
file = drive.CreateFile({'id': '1STxR_DQE0JCDpIE_9JzkgTTx4j-ga7YG'})
file.GetContentFile('annotations.zip')

In [0]:
!unzip data.zip >/dev/null
!unzip annotations.zip >/dev/null
!rm ./Data/56478665_421907071715102_3762114008077828096_n.jpg # this image has no object, so removing it (no xml exists for it)
!rm ./Data/55912462_267292044150469_7933255829271085056_n.jpg # this image has no object, so removing it (no xml exists for it)

In [0]:
import cv2
import glob
import xml.etree.ElementTree as ET
import os
import numpy as np
from matplotlib import pyplot as plt
import imutils
from PIL import Image

from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape, Flatten, Dense, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

import pandas as pd
import random
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
annot_dir = './Annotations/'
img_shape = 224

In [0]:
images = []
labels = [] # labels are [xmin, ymin, xmax, ymax]
file_list = list(glob.glob('./Data/*'))
for file in file_list:
  img = cv2.imread(file)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (img_shape, img_shape))
  tree = ET.parse(os.path.join(annot_dir, file.split('/')[-1])[:-3] + 'xml')
  root = tree.getroot()
  size = root.find('size')
  width = int(size.find('width').text)
  height = int(size.find('height').text)
  box = (root.find('object')).find('bndbox')
  xmin = int(box[0].text)
  ymin = int(box[1].text)
  xmax = int(box[2].text)
  ymax = int(box[3].text)
  
  if(width > height):
    img = imutils.rotate_bound(img, 90)
    xmin, ymin = height-ymin, xmin
    xmax, ymax = height-ymax, xmax    
  
  images.append(img)
  labels.append([xmin, ymin, xmax, ymax])
images = np.array(images)
images = np.stack(3*[images], axis=3)
labels = np.array(labels)

In [0]:
datagen = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, brightness_range=(0.7,1.3), zoom_range=0.4, 
                   fill_mode='reflect', horizontal_flip=True, vertical_flip=True, rescale=1/255, validation_split=0.1)

In [0]:
flow = datagen.flow(images, labels, batch_size=1)

In [0]:
images, labels = flow.next()
cv2.rectangel

In [0]:
x[1]

array([[630, 602, 382, 998]])

In [0]:
# class DataGenerator(Sequence):

#     def __init__(self, images, labels, batch_size, img_size=224, val=False):
      
#         super(DataGenerator, self).__init__()
      
#         self.data = images
#         self.labels = labels
#         self.batch_size = batch_size
#         self.index = 0
#         self.hor_flip = RandomHorizontalFlip(0.5)
#         self.ver_flip = RandomVerticalFlip(0.5)

#     def __len__(self):
#         return int(len(self.data)/self.batch_size)

#     def __getitem__(self, idx):
        
#         batch_images = []
#         for i in range(self.batch_size):
#           batch_images.append(images[index])
#           index = (index+1)%len(self.images)
        
#         # batch_images [batch_size, img_shape, img_shape, 3]
#         # batch_coords [batch_size, 4] - [x,y,w,h] - x,y is top left corner
#         return batch_images, batch_coords

In [0]:
def iou_metric(y_true, y_pred):
    
    gt = y_true
    pred = y_pred

    diff_width = K.minimum(gt[:,2], pred[:,2]) - K.maximum(gt[:,0], pred[:,0])
    diff_height = K.minimum(gt[:,3], pred[:,3]) - K.maximum(gt[:,1], pred[:,1])
    intersection = K.maximum(diff_width, 0) * K.maximum(diff_height, 0)

    area_gt = (gt[:,2]-gt[:,0]) * (gt[:,3]-gt[:,1])
    area_pred = (pred[:,2]-pred[:,0]) * (pred[:,3]-pred[:,1])
    union = K.maximum(area_gt + area_pred - intersection, 0)
    
    iou = K.round(intersection / (union + epsilon()))
    
    return iou

In [0]:
def create_model(img_size=224, trainable=False):
    model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False)
    
#     to freeze layers
    for layer in model.layers:
        layer.trainable = trainable
        
    for layer in model.layers:
      if layer.name == 'block_7_project_BN':
        x = layer.output
        break
    
#     regulariz = regularizers.l2(1)
    regulariz = None
        
    x = Conv2D(64, kernel_size=3, name="new_conv", activation='relu', kernel_regularizer=regulariz)(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, kernel_size=3, name="new_conv1", activation='relu', padding='same', kernel_regularizer=regulariz)(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, kernel_size=3, name="new_conv2", activation='relu', padding='same', kernel_regularizer=regulariz)(x)
    x = Conv2D(128, kernel_size=3, name="new_conv3", activation='relu', kernel_regularizer=regulariz)(x)
    x = BatchNormalization()(x)
    x = Conv2D(8, kernel_size=1, name="new_conv4", activation='relu', kernel_regularizer=regulariz)(x)
    x = Flatten()(x)
    pred = Dense(4)(x)

    return Model(inputs=model.input, outputs=pred)

In [0]:
model = create_model(img_size=img_shape)
# model.summary()

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 0s 0us/step


In [0]:
# from google.colab import files
# uploaded = files.upload()
# for k, v in uploaded.items():
#   open(k, 'wb').write(v)
# list(uploaded.keys())

In [0]:
lr = 1e-4
epochs=5000

In [0]:
model.metrics_names

[]

In [0]:
model.compile(loss="mean_squared_error", optimizer= optimizers.Adam(lr=lr), metrics=[iou_metric])

checkpoint = ModelCheckpoint("model-{iou_metric:.2f}.h5", monitor="iou_metric", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="max", period=5)
# stop = EarlyStopping(monitor="iou_metric", patience=5, mode="max")
# reduce_lr = ReduceLROnPlateau(monitor="iou_metric", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")

model.fit_generator(generator=datagen.flow(images, labels, batch_size=8),
                    epochs=epochs,
                    callbacks=[checkpoint],
                    shuffle=True,
                    verbose=1)


Epoch 1/5000
7/7 [==============================] - 1s 134ms/step - loss: 10403.8493 - iou_metric: 0.0364
Epoch 2/5000
7/7 [==============================] - 1s 114ms/step - loss: 10184.4912 - iou_metric: 0.0909
Epoch 3/5000
7/7 [==============================] - 1s 121ms/step - loss: 10468.9787 - iou_metric: 0.0727
Epoch 4/5000
7/7 [==============================] - 1s 130ms/step - loss: 9751.9563 - iou_metric: 0.1273
Epoch 5/5000
6/7 [========================>.....] - ETA: 0s - loss: 9837.8554 - iou_metric: 0.0851 
Epoch 00005: iou_metric improved from -inf to 0.09091, saving model to model-0.09.h5
7/7 [==============================] - 1s 140ms/step - loss: 9322.4354 - iou_metric: 0.0909
Epoch 6/5000
7/7 [==============================] - 1s 112ms/step - loss: 9939.3928 - iou_metric: 0.1091
Epoch 7/5000
7/7 [==============================] - 1s 119ms/step - loss: 11072.4038 - iou_metric: 0.0909
Epoch 8/5000
7/7 [==============================] - 1s 127ms/step - loss: 16005.9202 - io

In [0]:
# for i in range(50):
#   img = images[i]
#   cv2.rectangle(img, (labels[i][0], labels[i][1]),(labels[i][2], labels[i][3]),(0,255,0),3)
#   plt.imshow(img)
#   plt.show()

In [0]:
# class repeat_channel(image_gen):
#   def __init__(self, image_gen):
#     super(repeat_channel, self).__init__()
#     self.gen = image_gen
    
#   def __len__(self):
#     return 141
  
#   def __getitem__(self, idx):
#     img, y = self.gen.next()
#     return (np.concatenate([img,img,img], axis=3), y)